In [1]:
import pandas as pd
import numpy as np

In [2]:
import json, os, time
from tqdm.notebook import tqdm_notebook
with open('/Users/Lindsey/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [3]:
FOLDER = "..//Data/Extraction Data"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)


['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_results_tmdb.csv.gz']

In [4]:
basics = pd.read_csv("..//Data/Table Files/basics_df.csv.gz")

In [5]:
years_to_get = [2015, 2016, 2017, 2018, 2019, 2020]
years_to_get

[2015, 2016, 2017, 2018, 2019, 2020]

In [6]:
def read_and_fix_json(JSON_FILE):
   
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    except:
        
        with open(JSON_FILE,'r+') as f:
            bad_json = f.read()
            
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            final_char = final_brackets[first_char]
            
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            f.seek(0)
            f.write(good_json)
           
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

In [7]:
def get_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [8]:
def write_json(new_data, filename):
    with open(filename,'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file)

In [9]:
# Start of OUTER loop
for YEAR in tqdm_notebook(years_to_get, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = read_and_fix_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]        

    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {YEAR}', position=1, leave=True):
        try:
            temp = get_rating(movie_id)
            write_json(temp, JSON_FILE)
            time.sleep(0.02)
            
        except Exception as e:
            continue
            
    final_year_df_2 = pd.read_json(JSON_FILE)
    final_year_df_2.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/6 [00:00<?, ?it/s]

Movies from 2015:   0%|          | 0/4884 [00:00<?, ?it/s]

Movies from 2016:   0%|          | 0/5147 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/5506 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5631 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5733 [00:00<?, ?it/s]

Movies from 2020:   0%|          | 0/4870 [00:00<?, ?it/s]